In [39]:
import pandas as pd
import os
import numpy as np
import pprint

In [3]:
anno = pd.read_csv("../../../sample_annotation/Data/sample_annotation.tsv", sep='\t')

In [4]:
# print(anno['FIBROBLAST_ID'])
# anno.iloc[1]
anno.loc[1,"FIBROBLAST_ID"]

'100416'

In [4]:
anno_mae = anno[anno["LAB"] == "PROKISCH"] 

In [5]:
np.unique(anno_mae["LAB"])
anno_mae.shape

(1268, 53)

In [6]:
#np.isnan(anno_mae.EXOME_ID)
anno_mae = anno_mae[pd.notnull(anno_mae.EXOME_ID)]
anno_mae = anno_mae[pd.notnull(anno_mae.RNA_ID)]

In [7]:
anno_mae.shape

(284, 53)

In [8]:
anno_mae = anno_mae[["EXOME_ID", "RNA_ID"]]

In [9]:
anno_mae['rna_file'] = ["/s/project/mitoMultiOmics/raw_data/helmholtz/" + x + "/RNAout/paired-endout/stdFilenames/" + x + ".bam" for x in anno_mae["RNA_ID"]]

In [10]:
anno_mae['vcf_file'] = ["/s/project/mitoMultiOmics/raw_data/helmholtz/" + x + "/exomicout/paired-endout/stdFilenames/" + x + ".vcf.gz" for x in anno_mae["EXOME_ID"]]

In [12]:
# anno_mae["vcf_file"]
anno_mae['vcf_exists'] = [os.path.exists(x) for x in anno_mae["vcf_file"]]
anno_mae['rna_exists'] = [os.path.exists(x) for x in anno_mae["rna_file"]]
anno_mae = anno_mae[anno_mae['vcf_exists'] & anno_mae['rna_exists']]

In [40]:
# ('mae/' + anno_mae["EXOME_ID"] + '-' + anno_mae["RNA_ID"] + '.Rds').tolist()

## Complete Function

In [9]:
def mae_files(anno):
    
    # subset and clean
    anno_mae = anno[anno["LAB"] == "PROKISCH"]
    anno_mae = anno_mae[pd.notnull(anno_mae.EXOME_ID)]
    anno_mae = anno_mae[pd.notnull(anno_mae.RNA_ID)]
    anno_mae = anno_mae[["EXOME_ID", "RNA_ID"]]

    # create file names
    anno_mae['rna_file'] = ["/s/project/mitoMultiOmics/raw_data/helmholtz/" + x + "/RNAout/paired-endout/stdFilenames/" + x + ".bam" for x in anno_mae["RNA_ID"]]
    anno_mae['vcf_file'] = ["/s/project/mitoMultiOmics/raw_data/helmholtz/" + x + "/exomicout/paired-endout/stdFilenames/" + x + ".vcf.gz" for x in anno_mae["EXOME_ID"]]

    # check for missing files
    anno_mae['vcf_exists'] = [os.path.exists(x) for x in anno_mae["vcf_file"]]
    anno_mae['rna_exists'] = [os.path.exists(x) for x in anno_mae["rna_file"]]
    anno_mae = anno_mae[anno_mae['vcf_exists'] & anno_mae['rna_exists']]
    
    vcf = anno_mae["EXOME_ID"] 
    rna = anno_mae["RNA_ID"]
    
    return vcf.tolist(), rna.tolist()

In [10]:
vcf, rna = mae_files(anno)

# Test for all vcf files

In [50]:
def all_vcf(anno):

    # subset and clean
    anno = anno[anno["LAB"] == "PROKISCH"]
    anno = anno[pd.notnull(anno.EXOME_ID)]
    anno_vcf = anno[["EXOME_ID"]].copy()

    anno_vcf['file'] = ["/s/project/mitoMultiOmics/raw_data/helmholtz/" + x + "/exomicout/paired-endout/stdFilenames/" + x + ".vcf.gz" for x in anno_vcf["EXOME_ID"]]
    anno_vcf['vcf_exists'] = [os.path.exists(x) for x in anno_vcf["file"]]
    anno_vcf = anno_vcf[anno_vcf['vcf_exists']]
    
    return pd.unique(anno_vcf["EXOME_ID"]).tolist()

In [51]:
all_vcfs = all_vcf(anno)
len(all_vcfs)

752

In [56]:
len(vcf)

232

## OUTRIDER group files

In [2]:
anno = pd.read_csv("../../../sample_annotation/Data/sample_annotation.tsv", sep='\t')

In [64]:
def outrider_files(anno):

    # subset and clean
    anno_outrider = anno[(anno.LAB == "PROKISCH") & pd.notnull(anno.RNA_ID) & pd.notnull(anno.OUTRIDER_GROUP)]
    anno_outrider = anno_outrider[["RNA_ID", "OUTRIDER_GROUP"]].drop_duplicates().copy()

    # create filenames and ignore missing files
    anno_outrider['file'] = ["/s/project/mitoMultiOmics/raw_data/helmholtz/" + x +
                             "/RNAout/paired-endout/stdFilenames/" + x + ".bam"
                             for x in anno_outrider["RNA_ID"]]
    anno_outrider['file_exists'] = [os.path.exists(x) for x in anno_outrider["file"]]
    anno_outrider = anno_outrider[anno_outrider['file_exists']]

    # subset by OUTRIDER_GROUP
    outrider_groups = []
    for s in set(anno_outrider.OUTRIDER_GROUP):
        outrider_groups.extend(s.split(','))
    outrider_ids = {og : anno_outrider.loc[anno_outrider.OUTRIDER_GROUP.str.contains('(^|,)' + og), 'RNA_ID'].tolist() for og in set(outrider_groups)}
    return {og: _list for og, _list in outrider_ids.items() if len(_list) > 40}

In [65]:
anno.head()

,FIBROBLAST_ID,EXOME_ID,RNA_ID,GENOME_ID,KNOWN_MUTATION,TISSUE,DISEASE,MAX_RNA_TO_EXOME_IDENTITY,PROTEOME_ID,GENDER,...,PubMed_ID,Last_revised,fib_id,PATIENT_ID,PEDIGREE_new,g0,g1,OUTRIDER_GROUP,g2,g3
0,100416,NaN,MUC3196,NaN,NaN,FIBROBLAST,HEALTHY,0.41,NaN,NaN,...,NaN,NaN,100416,NaN,NaN,all,fib_ss,"all,fib_ss,fib,no_blood",fib,no_blood
1,100416,NaN,MUC3197,NaN,NaN,FIBROBLAST,HEALTHY,0.41,NaN,NaN,...,NaN,NaN,100416,NaN,NaN,all,gal,"all,gal,fib,no_blood",fib,no_blood
2,100634,92582,100634R,NaN,NaN,FIBROBLAST,MITO,0.94,P100634,female,...,NaN,20181119.0,100634,p67077,ExDiag0510,all,fib_ss,"all,fib_ss,fib,no_blood",fib,no_blood
3,100782,98288,110468R,NaN,NaN,FIBROBLAST,MITO,NaN,P100782,male,...,NaN,NaN,100782,p70754,ExDiag0874,all,fib_ss,"all,fib_ss,fib,no_blood",fib,no_blood
4,101309,104905,110442R,NaN,NaN,FIBROBLAST,MITO,NaN,NaN,female,...,NaN,NaN,101309,p72867,SALK0009,all,fib_ss,"all,fib_ss,fib,no_blood",fib,no_blood


In [66]:
of = outrider_files(anno)

/opt/modules/i12g/anaconda/3-4.1.1/envs/genetic_diagnosis/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [67]:
len(of['fib_ns'])

157